# Clustering Crypto

In [7]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [14]:
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [29]:
# Creating a DataFrame (which needed to be transposed)
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()


,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,FullName,FullyPremined,Id,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,154593,0,0,N/A,42 Coin,1427211129,42 Coin (42),0,4321,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,300 token (300),0,749869,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,365Coin (365),0,33639,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview
404,Scrypt,287048,0,60,N/A,404Coin,1466100361,404Coin (404),0,21227,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,602,False,404,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",532000000,N/A,0,/coins/404/overview
433,N/A,9187775,2,0,7605,433 Token,1541597321,433 Token (433),0,926547,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0x738505a5f31bf72e0b70298bca81150eb1b7c751,3505,False,433,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",1000000000,N/A,1.09083e+08,/coins/433/overview


### Data Preprocessing

In [32]:
# Remove unnecessary columns
slim_crypto_df = crypto_df.drop(columns=['TotalCoinsFreeFloat', 'Url', 'Taxonomy', 'SortOrder', 'Sponsored', 'SmartContractAddress', 'Id', 'BuiltOn', 'FullName', 'ImageUrl', 'ContentCreatedOn', 'PreMinedValue'])


In [33]:
# Keep only cryptocurrencies that are on trading
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['IsTrading'] == True]


In [34]:
# Keep only cryptocurrencies with a working algorithm
slim_crypto_df = slim_crypto_df.loc[slim_crypto_df['Algorithm'] == True]


In [35]:
# Remove the "IsTrading" column
slim_crypto_df = slim_crypto_df.drop(columns='IsTrading')


In [36]:
# Removing rows with at least 1 null value
slim_crypto_df = slim_crypto_df.dropna()


In [ ]:
# Removing rows with cryptocurrencies without coins mined



In [ ]:
# Dropping rows where there are 'N/A' text values



In [ ]:
# Fetching the cryptocurrencies names prior to drop them from crypto_df



In [ ]:
# Removig the cryptocurrency name since it's not going to be used on the clustering algorithm



In [ ]:
# Creating dummies variables for text features



In [ ]:
# Standardizing data



### Reducing Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to 3 principal components



In [ ]:
# Creating a DataFrame with the principal components data



### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:


# Create the Elbow Curve using hvPlot



Running K-Means with `k=<you best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Creating a new DataFrame including predicted clusters and cryptocurrencies features



### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters



#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos



#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scaling data to create the scatter plot



In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

